# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0901, -0.1413,  0.0145, -0.0741, -0.0530,  0.1715,  0.0482,  0.2100,
          0.0972,  0.2370],
        [-0.0008, -0.1410,  0.0156, -0.0534, -0.0940,  0.1948,  0.0733,  0.3680,
          0.1152,  0.3197]], grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.0617, -0.1105, -0.2544,  0.1798,  0.1566, -0.1288,  0.0832,  0.1959,
         -0.1733, -0.1143],
        [ 0.1267, -0.0508, -0.4114,  0.2968,  0.2962, -0.0076,  0.2384,  0.3175,
         -0.1889, -0.1887]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0127,  0.1492, -0.2550, -0.0494, -0.2078,  0.3152, -0.0481,  0.0901,
          0.0065, -0.0505],
        [-0.0771,  0.2180, -0.3476, -0.0347, -0.1752,  0.1750, -0.0380,  0.0401,
          0.0966,  0.0465]], grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1983, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2429, grad_fn=<SumBackward0>)